# Faultproof Withdrawals Triage Runbook
In order to start this runbook locally please use the README located at [https://github.com/ethereum-optimism/monitorism/blob/main/op-monitorism/faultproof_withdrawals/runbooks/automated/README.md](https://github.com/ethereum-optimism/monitorism/blob/main/op-monitorism/faultproof_withdrawals/runbooks/automated/README.md)

General instructions to start the runbook locally:
```bash
cd op-monitorism/faultproof_withdrawals/runbooks/automated/
make start
```


In [ ]:
from dotenv import load_dotenv
import os
from web3 import Web3
from lib.superchain import *
from lib.web3 import *
from pprint import pprint
from datetime import datetime, timedelta


#parameters setup (there should be no need to change the one below)
abi_folder_path="abi"

env_file = ".env"
if os.path.exists(env_file):
    load_dotenv(env_file)
    print("Environment variables loaded from .env file.")
else:
    print("No .env file found. Using system environment variables. Make sure to set them up. An example .env file is provided insite the automated folder .env.example")

L1_GETH_URL = os.getenv('L1_GETH_URL')
L2_OP_NODE_URL = os.getenv('L2_OP_NODE_URL')
L2_OP_GETH_URL = os.getenv('L2_OP_GETH_URL')

## setup 
In order to proceed make sure you set the parameters below.
The url used below are the trusted nodes that are going to be used.

In [23]:
#set ignore_certificate if you are using a local https
ignore_url_certificate=True

#set cahin you need to monitor
l1_chain_name="mainnet"
l2_chain_name="op"


## Loading local values and superchain regsitry values

In [ ]:
eth_scan_url="https://etherscan.io"

if l1_chain_name=="sepolia":
    eth_scan_url="https://sepolia.etherscan.io"
    
superchain=get_superchain_file(l1_chain_name, l2_chain_name)
OptimismPortalProxy=superchain["addresses"]["OptimismPortalProxy"]

print(f"OptimismPortal2 address:{eth_scan_url}/address/{OptimismPortalProxy}#readProxyContract")

web3_utility=Web3Utility(L1_GETH_URL, L2_OP_GETH_URL,L2_OP_NODE_URL,abi_folder_path, OptimismPortalProxy, ignore_certificate=ignore_url_certificate)

# Triage alert: faultproof-withdrawal-forgery-detection-stalled
This alert monitors the number of withdrawal events that are considered normal in a chain. If the number of withdrawal events goes below a specified threshold, we trigger this alert.

We want to verify when it was last withdrawals on the chain. If there are no provenwithdrawals events on the chain in the last day, then we know the alert is a false positive and there is no issue with the monitor.

In [ ]:
result=web3_utility.find_latest_withdrawal_event()

log=result["log"]
timestamp=result["timestamp"]

print(f"Last withdrawal event:\ntimestamp: {timestamp["timestamp"]} \ntimestamp: {timestamp["formatted_timestamp"]}\n")
print(f"Last withdrawal happened: \nat block: {eth_scan_url}/block/{log["blockNumber"]} \nwith transaction hash: {eth_scan_url}/tx/0x{log["transactionHash"].hex()}")
print(f"Logs can be found at: {eth_scan_url}/address/{OptimismPortalProxy}#events using filter:0x798f9f13695f8f045aa5f80ed8efebb695f3c7fe65da381969f2f28bf3c60b97")

# Convert the Unix timestamp to a datetime object
event_time = datetime.fromtimestamp(timestamp["timestamp"])

# Check if the event timestamp is more than 24 hours ago
if event_time < (datetime.now() - timedelta(hours=24)):
    print("\nLast withdrawal event happened more than 24 hours ago")
else:
    print("\nWARNING: Last withdrawal event happened less than 24 hours ago. The monitoring may be stalled")
